In [1]:
import pymorphy2


In [2]:
morph = pymorphy2.MorphAnalyzer()

In [3]:
morph.parse('стали')

[Parse(word='стали', tag=OpencorporaTag('VERB,perf,intr plur,past,indc'), normal_form='стать', score=0.984662, methods_stack=((<DictionaryAnalyzer>, 'стали', 904, 4),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 1),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 2),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,loct'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 5),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,nomn'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 6),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,accs'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 9),))]

In [4]:
morph.parse('cтали')[0].inflect({'nomn'})

Parse(word='cталь', tag=OpencorporaTag('NOUN,inan,femn sing,nomn'), normal_form='cталь', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'таль', 13, 0), (<UnknownPrefixAnalyzer>, 'c')))

In [5]:
#lec = 'D:\\MagesGuild\\Нейропрепод\\ontology\\Prolog_lec.txt'
lec = 'Prolog_lec.txt'
kb = []
with open(lec, 'r', encoding='utf-8') as reader:
    for line in reader:
        line = line.strip()
        if len(line)==0 or ':-' in line:
            continue
        pred = line[:line.index('(')]
        args = line[len(pred)+1:-2].replace("'", '').split(',')
        kb.append((pred, args))
        

In [6]:
def inflect(word, g):
    words = word.split(' ')
    return ' '.join([morph.parse(w)[0].inflect(g).word for w in words])

In [7]:
def multi_inflect(words, g):
    if len(words) == 1:
        return inflect(words[0], g)
    inflected_words = [inflect(w, g) for w in words]
    return ', '.join(inflected_words[:-1])+' и '+inflected_words[-1]

In [8]:
def explain(term, kb):
    out = ''
    filt = {}
    for p, a in kb:
        if a[0]==term:
            filt[p] = filt.get(p, [])+[a]
        elif term in a:
            filt['inv_'+p] = filt.get('inv_'+p, [])+[a]
    out += term
    if 'syn' in filt:
        out += ' ('+', '.join([s[1] for s in filt['syn']]) + '). '
    else:
        out += '. '
    if 'inv_hyp' in filt:
        out += '{} -- это вид {}. '.format(term, multi_inflect([s[0] for s in filt['inv_hyp']], {'gent'}))
    if 'hyp' in filt:
        if len(filt['hyp']) >= 1:
            out += '{} подразделяются на {}. '.format(inflect(term, {'plur'}).title(), multi_inflect([s[1] for s in filt['hyp']], {'accs', 'plur'}))
    if 'part' in filt:
        out += '{} входит в состав {}. '.format(term, multi_inflect([s[1] for s in filt['part']], {'gent'}))
    if 'inv_part' in filt:
        if len(filt['inv_part']) >= 1:
            out += '{} состоит из {}. '.format(term, multi_inflect([s[0] for s in filt['inv_part']], {'gent', 'plur'}))
    if 'attr' in filt:
        for a in filt['attr']:
            out += '{} {}. '.format(term, a[1])
    if 'ekz' in filt:
        if len(filt['ekz']) > 1:
            out += 'Примеры {} -- {}. '.format(inflect(term, {'gent', 'plur'}), multi_inflect([s[1] for s in filt['ekz']], set()))
        else:
            out += 'Пример {} -- {}. '.format(inflect(term, {'gent', 'sing'}), multi_inflect([s[1] for s in filt['ekz']], set()))
        
        
    print(out)

In [9]:
inflect('искусственый язык',{'accs', 'plur', 'inan'})

'искусственые языки'

In [10]:
morph.parse('язык')

[Parse(word='язык', tag=OpencorporaTag('NOUN,inan,masc sing,accs'), normal_form='язык', score=0.90909, methods_stack=((<DictionaryAnalyzer>, 'язык', 3101, 3),)),
 Parse(word='язык', tag=OpencorporaTag('NOUN,anim,masc sing,nomn'), normal_form='язык', score=0.045454, methods_stack=((<DictionaryAnalyzer>, 'язык', 2, 0),)),
 Parse(word='язык', tag=OpencorporaTag('NOUN,inan,masc sing,nomn'), normal_form='язык', score=0.045454, methods_stack=((<DictionaryAnalyzer>, 'язык', 3101, 0),))]

In [11]:
explained = {}

In [12]:
for pred, attr in kb:
    if pred == 'term' and attr[0] not in explained:
        explain(attr[0], kb)

Естественный язык (ЕЯ). Естественный язык -- это вид языка. Естественный язык  используется для общения людей. Естественный язык  не создан целенаправленно. Примеры естественных языков -- русский, английский, китайский и казахский. 
Формальный язык. Формальный язык -- это вид языка. Формальные Языки подразделяются на языки программирования и регулярных языки. Формальный язык  создан целенаправленно. Формальный язык  не используется для общения людей. Пример формального языка -- язык математической логики. 
Искусственный язык. Искусственный язык -- это вид языка. Искусственные Языки подразделяются на эльфийских языки. Искусственный язык  используется для общения людей. Искусственный язык  создан целенаправленно. Примеры искусственных языков -- эсперанто, ложбан и токипона. 
Язык. Языки подразделяются на естественных языки, формальных языки и искусственных языки. 
Регулярный язык. Регулярный язык -- это вид формального языка. Примеры регулярных языков -- язык порождённый регулярными выра

In [13]:
kb

[('term', ['Естественный язык']),
 ('term', ['Формальный язык']),
 ('term', ['Искусственный язык']),
 ('term', ['Язык']),
 ('term', ['Регулярный язык']),
 ('term', ['Эльфийский язык']),
 ('term', ['Обработка естественного языка']),
 ('term', ['Научное направление']),
 ('term', ['Анализ языка']),
 ('term', ['Синтез языка']),
 ('term', ['Текст']),
 ('term', ['Уровень разбиения текста']),
 ('term', ['Фонологический уровень']),
 ('term', ['Уровен символов']),
 ('term', ['Морфологический уровень']),
 ('term', ['Синтаксический уровень']),
 ('term', ['Уровень фраз']),
 ('term', ['Уровень текста']),
 ('term', ['Текстовая единица']),
 ('term', ['Символ']),
 ('term', ['Фонема']),
 ('term', ['Слово']),
 ('term', ['Словоформа']),
 ('term', ['Фраза']),
 ('term', ['Предложение']),
 ('term', ['Этап обработки']),
 ('term', ['Графематический анализ']),
 ('term', ['Морфологический анализ']),
 ('term', ['Фрагментационный анализ']),
 ('term', ['Синтаксический анализ']),
 ('term', ['Семантический анализ'])

In [15]:
#inputo = 'D:\\MagesGuild\\Нейропрепод\\ontology\\lec_formal.txt'
inputo = 'lec_formal.txt'
kb_gen = []
lines = []
for l in open(inputo, 'r', encoding='utf-8'):
    lines += l.strip().split('. ')

In [24]:
morph.parse('по')[0].tag.POS

'PREP'

In [195]:
def read_ng(words):
    ng = []
    case = 'nomn'
    for i, w in enumerate(words):
        if len(w)==0 or not (w[0].isalpha()):
            i-=1
            break
        if 'LATN' in morph.parse(w)[0].tag:
            ng += [w]
            continue
        pos = morph.parse(w)[0].tag.POS
        if pos in ['NOUN', 'ADJF', 'PRTF', "PREP"]:
            if pos=='PRTF':
                case = 'nomn'
            ng += [inflect(w, set([case, 'sing']))]
            if pos=='NOUN':
                case = 'gent'
            if pos=='PRTF':
                case = 'ablt'
        else:
            i-=1
            break
    return ng, words[i+1:]

In [196]:
def multiparse(words):
    words = ' '.join(words).replace(' и ', ', ').split(', ')
    return [read_ng(w.split(' '))[0] for w in words]

In [197]:
def read_brackets(words):
    ret = []
    line = ' '.join(words).replace(') ', ')')
    ret = line[line.index('(')+1: line.index(')')].split(' ')
    res = line[line.index(')')+2:]
    return ret, res

In [225]:
def parse(line):
    ret = []
    words = line.replace('.', '').split(' ')
    if words[0] in ['Пример', 'Примеры']:
        root, res = read_ng(words[1:])
        ret.append(('term', (root)))
        args = multiparse(res[1:])
        for a in args:
            ret.append(('ekz', (root, a)))
        return ret
    ng, res = read_ng(words)
    ret.append(('term', (ng)))
    if len(res) and res[0].startswith('('):
        syns, res = read_brackets(res)
        for s in multiparse(syns): 
            ret.append(('syn', (ng, s)))
    if '-- это вид ' in line:
        arg = read_ng(res[3:])[0]
        ret.append(('hyp', (arg, ng)))
    elif 'подразделяются на ' in line:
        args = multiparse(res[2:])
        for a in args:
            ret.append(('hyp', (ng, a)))
    elif 'состоит из' in line:
        args = multiparse(res[2:])
        for a in args:
            ret.append(('part', (ng, a)))
    elif 'входит в состав ' in line:
        arg = read_ng(res[3:])[0]
        ret.append(('part', (arg, ng)))
    elif len(res) and len(res[0]):
        ret.append(('attr', (ng, res)))
    return ret #' '.join(ng)


In [226]:
read_ng(['Putin', 'Shumtin'])

(['Putin', 'Shumtin'], [])

In [227]:
kb_gen = []
for line in lines:
    kb_gen+=[parse(line)]

In [228]:
kb_gen

[[('term', ['естественный', 'язык']),
  ('syn', (['естественный', 'язык'], ['ея']))],
 [('term', ['естественный', 'язык']),
  ('hyp', (['язык'], ['естественный', 'язык']))],
 [('term', ['естественный', 'язык'])],
 [('term', ['естественный', 'язык'])],
 [('term', ['естественный', 'язык']),
  ('ekz', (['естественный', 'язык'], ['русский'])),
  ('ekz', (['естественный', 'язык'], ['английский'])),
  ('ekz', (['естественный', 'язык'], ['китайский'])),
  ('ekz', (['естественный', 'язык'], ['казахский']))],
 [('term', ['формальный', 'язык'])],
 [('term', ['формальный', 'язык']),
  ('hyp', (['язык'], ['формальный', 'язык']))],
 [('term', ['формальный', 'язык']),
  ('hyp', (['формальный', 'язык'], ['язык', 'программирования'])),
  ('hyp', (['формальный', 'язык'], ['регулярный', 'язык']))],
 [('term', ['формальный', 'язык'])],
 [('term', ['формальный', 'язык'])],
 [('term', ['формальный', 'язык']),
  ('ekz', (['формальный', 'язык'], ['язык', 'математической', 'логики']))],
 [('term', ['искусстве